In [1]:
import pandas as pd
import numpy as np
from gremlin_python.structure.graph import Graph
from gremlin_python.process.traversal import Cardinality
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

In [4]:
#Assumes gremlin-server is running at localhost:8182; likely to change in future
g = Graph().traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

In [66]:
g.V().drop().iterate();

city_frame=pd.read_csv("citiesUS.csv")
city_frame.set_index("City (US)",inplace=True)

for index,series in city_frame.iterrows():
    if g.V().hasLabel("Admin1").has("ID",series["State"]).toList():
        pass;
    else:
        g.addV("Admin1").property("ID",series["State"]).next()
        
    if (g.V().hasLabel("Admin1").has("ID",series["State"]).out()
                .hasLabel("Admin2").has("ID",series["Admin2"]).toList()):
        county=(g.V().hasLabel("Admin1").has("ID",series["State"]).out()
                .hasLabel("Admin2").has("ID",series["Admin2"]).next())
    else:
        county=(g.addV("Admin2").property("ID",series["Admin2"]).
             property("name",series["Admin2 Name"]).next())
        state=g.V().hasLabel("Admin1").has("ID",series["State"]).next()
        g.V(state).addE("Devolution").to(county).next()
        g.V(county).addE("Centralization").to(state).next()
        
    city=(g.addV("City").property("name",index).next())
    g.V(county).addE("Devolution").to(city).next()
    g.V(city).addE("Centralization").to(county).next()
    
    series_dict=series.drop(labels=["Admin2", "Admin2 Name","State"]).to_dict()
    
    for key,value in series_dict.items():
        if (key=="Longitude" or key=="Latitude"):
            value=round(float(value),4)
        g.V(city).property(key,value).next()

#g.V().drop().iterate();